In [2]:
import numpy 
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from collections import Counter

import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer

In [3]:
data = pd.read_csv('./train.csv', encoding='latin-1', delimiter=',', quotechar='"')
print(data.shape)
data.head()

(99781, 3)


,ItemID,Sentiment,SentimentText;;;;;;;;
0,1,0.0,is so sad for my APL frie...
1,2,0.0,I missed the New Moon trail...
2,3,1.0,omg its already 7:30 :O;;;;;;;;
3,4,0.0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0.0,i think mi bf is cheating on me!!! ...


In [4]:
data = data.dropna()

In [5]:
# Obtener la distribución de clases
class_distribution = data['Sentiment'].value_counts()

# Imprimir la distribución de clases
print(class_distribution)

1.0    41615
0.0    33424
Name: Sentiment, dtype: int64


In [6]:
minority_class_size = min(class_distribution)

# Undersampling de la clase mayoritaria
undersampled_df = pd.concat([
    data[data['Sentiment'] == 0],
    data[data['Sentiment'] == 1].sample(minority_class_size, replace=True)
])

new_class_distribution = undersampled_df['Sentiment'].value_counts()

In [17]:
data = undersampled_df

In [18]:
class_distribution = data['Sentiment'].value_counts()

In [19]:
class_distribution

0.0    33424
1.0    33424
Name: Sentiment, dtype: int64

In [9]:
data =  data.rename(columns={'SentimentText;;;;;;;;': 'SentimentText'})
data['Sentiment'] = data['Sentiment'].astype(int)


#Limpieza de textos

In [10]:
for index, row in data.iterrows():
    text = row['SentimentText']
    lower_case = text.lower()
    cleaned_text = lower_case.translate(
        str.maketrans('', '', string.punctuation))
    data.at[index, 'SentimentText'] = cleaned_text

data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my apl friend
1,2,0,i missed the new moon trailer
2,3,1,omg its already 730 o
3,4,0,omgaga im sooo im gunna cry ive be...
4,5,0,i think mi bf is cheating on me tt


In [11]:
data.columns

Index(['ItemID', 'Sentiment', 'SentimentText'], dtype='object')

<h2>Modelo 1 - Regresión logística

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Dividir los datos en características (X) y etiquetas (y)
X = data['SentimentText']
y = data['Sentiment']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un vectorizador TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Crear y entrenar el modelo de Regresión Logística
logreg_model = LogisticRegression()
logreg_model.fit(X_train_vectorized, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = logreg_model.predict(X_test_vectorized)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.70      0.73      6729
           1       0.77      0.82      0.79      8279

    accuracy                           0.77     15008
   macro avg       0.77      0.76      0.76     15008
weighted avg       0.77      0.77      0.76     15008



c:\Users\Cristian Aguirre\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
text = "I can't believe how poorly this game is designed. The controls are clunky, the graphics are outdated, and the gameplay is incredibly boring. I regret wasting my money on it."
text = 'Im happy'
new_text_vectorized = vectorizer.transform([text])
predicted_sentiment = logreg_model.predict(new_text_vectorized)

print(predicted_sentiment)



[1]


<h2>Modelo 2 - SVM


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Dividir los datos en características (X) y etiquetas (y)
X = data['SentimentText']
y = data['Sentiment']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un vectorizador TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Crear y entrenar el modelo SVM
svm_model = SVC()
svm_model.fit(X_train_vectorized, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test_vectorized)

# Evaluar el desempeño del modelo
print(classification_report(y_test, y_pred))


KeyboardInterrupt: 

In [ ]:
text = "I can't believe how poorly this game is designed. The controls are clunky, the graphics are outdated, and the gameplay is incredibly boring. I regret wasting my money on it."
text = 'Im happy'

# Aplicar el preprocesamiento y transformación de características
new_text_vectorized = vectorizer.transform([text])

# Realizar la predicción
prediction = svm_model.predict(new_text_vectorized)

# Imprimir la predicción
print(prediction)

[1]
